In [2]:
import numpy as np
import pandas as pd
import cv2
import os

In [3]:
input_path = 'C:/SSD_Projects/Bengali_git/input/'
train_df_ = pd.read_csv(os.path.join(input_path,'train.csv'))
HEIGHT = 137
WIDTH = 236
img_size = 128

In [4]:
def read_data(nf):
    nf=int(nf)
    train_df = pd.read_feather(os.path.join(input_path,f'train_image_data_{nf}.feather'))
    return train_df
def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax
def resize(df, img_size = 128, pad=16):
    resized = {}
    for i in range(df.shape[0]):
        image = 255 - df.loc[df.index[i]].values.reshape(137,236)
        ymin,ymax,xmin,xmax = bbox(image[5:-5,5:-5] > 60)
        xmin = xmin - 13 if (xmin > 13) else 0
        ymin = ymin - 10 if (ymin > 10) else 0
        xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
        ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
        image = image[ymin:ymax,xmin:xmax]
        image[image < 28] = 0
        lx, ly = xmax-xmin,ymax-ymin
        l = max(lx,ly) + pad
        image = np.pad(image, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
        image = cv2.resize(image,(img_size,img_size))
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

In [4]:
for i in range(4):
    train_df = pd.merge(read_data(i), train_df_, on='image_id').drop(['image_id','grapheme'], axis=1)
    x_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
    x_train = np.divide(resize(x_train), 255.0, dtype = 'float16')
    x_train = x_train.values.reshape(-1, img_size, img_size, 1)
    np.save((os.path.join(input_path,f'train_image_data_{i}')),x_train)


